# Writing Algorithms
This notebook will allow a developer to:

*   Understand the basic structure of a Pythia policy.
*   Use the Designer API for simplfying algorithm design.




## Setting up our environment
Here, we install necessary packages and import required modules.

In [ ]:
# Install OSS Vizier.
!pip install google-vizier

In [ ]:
from typing import Optional, Sequence

from vizier import pythia
from vizier import pyvizier
from vizier import algorithms
from vizier._src.algorithms.policies import designer_policy
from vizier._src.algorithms.evolution import nsga2

## Pythia Policies
The Vizier server keeps a mapping from algorithm names to `Policy` objects. All algorithm implementations to be hosted on the server must eventually be wrapped into a `Policy`.

Every `Policy` is injected with a `PolicySupporter`, which is a client used for fetching data from the datastore. This design choice serves the core purposes:

1. The policy is effectively stateless, and thus can be deleted and recovered at any time (e.g. due to a server preemption or failure).
2. Consequently, this avoids needing to save an explicit and potentially complicated algorithm state. The "algorithm state" can be recovered purely from the entire study, containing metadata, the study config, and trials.

We show the `Policy` abstract class explicitly below. Exact code can be found [here](https://github.com/google/vizier/blob/main/vizier/_src/pythia/policy.py).

In [ ]:
class Policy(abc.ABC):
  """Interface for Pythia2 Policy subclasses."""

  @abc.abstractmethod
  def suggest(self, request: SuggestRequest) -> SuggestDecision:
    """Compute suggestions that Vizier will eventually hand to the user."""

  @abc.abstractmethod
  def early_stop(self, request: EarlyStopRequest) -> EarlyStopDecisions:
    """Decide which Trials Vizier should stop."""

  @property
  def should_be_cached(self) -> bool:
    """Returns True if it's safe & worthwhile to cache this Policy in RAM."""
    return False

## Example Pythia Policy
Here, we write a toy policy, where we only act on `CATEGORICAL` parameters for simplicity. The `make_parameters` function will simply for-loop over every category and then cycle back.

In [ ]:
def make_parameters(study_config: pyvizier.StudyConfig,
                    index: int) -> pyvizier.ParameterDict:
  parameter_dict = pyvizier.ParameterDict()
  for parameter_config in study_config.search_space.parameters:
    if parameter_config.type == pyvizier.ParamterType.CATEGORICAL:
      feasible_values = parameter_config.feasible_values
      categorical_size = len(feasible_values)
      parameter_dict[parameter_config.name] = pyvizier.ParameterValue(
          value=feasible_values[index % categorical_size])
    else:
      raise ValueError("This function only supports CATEGORICAL parameters.")
  return parameter_dict

To collect the `index` from the database, we will use the `PolicySupporter` to obtain all completed trials and look at the maximum trial ID.

In [ ]:
def get_next_index(policy_supporter: pythia.PolicySupporter):
  """Returns current trial index."""
  completed_trial_ids = [
      t.id for t in policy_supporter.GetTrials(
          status_matches=pyvizier.TrialStatus.COMPLETED)
  ]

  if completed_trial_ids:
    return max(completed_trial_ids)
  return 0

We can now put it all together into our Pythia Policy.

In [ ]:
class MyPolicy(pythia.Policy):
  def __init__(self, policy_supporter: pythia.PolicySupporter):
    self._policy_supporter = policy_supporter

  def suggest(self, request: pythia.SuggestRequest) -> pythia.SuggestDecision:
    """Gets number of Trials to propose, and produces Trials."""
    suggest_decision_list = []
    for _ in range(request.count):
      index = get_next_index(self._policy_supporter)
      parameters = make_parameters(request.study_config, index)
      suggest_decision_list.append(
          pyvizier.TrialSuggestion(parameters=parameters))
    return pythia.SuggestDecision(
        suggestions=suggest_decision_list, metadata=pyvizier.MetadataDelta())

## Designers

While Pythia policies are the proper interface for algorithms to be hosted on
the server, we also provide the `Designer` API, a simplified entry point for
implementing suggestion and early stopping algorithms.

The `Designer` interface is designed to let a developer forget about the
ultimate goal of serving the algorithm in a distributed environment. We may
pretend we're using the algorithm locally by doing a suggest-update loop in RAM,
during the lifetime of a study.

We display the `Designer` class below. Exact code can be found
[here](https://github.com/google/vizier/blob/main/vizier/_src/algorithms/core/abstractions.py).

In [ ]:
class Designer(_SuggestionAlgorithm):
  """Suggestion algorithm for sequential usage."""

  @abc.abstractmethod
  def update(self, delta: CompletedTrials) -> None:
    """Reflect the delta in the designer's state."""
    pass

To implement our same algorithm above in a Designer, we only need to implement the `update()` and `suggest()` methods using our previous `make_parameters` function. Note that the designer class can now store completed trials inside its `self._completed_trials` attribute.

In [ ]:
class MyDesigner(algorithms.Designer):

  def __init__(self, study_config: pyvizier.StudyConfig):
    self._study_config = study_config
    self._completed_trials = []

  def update(self, delta: algorithms.CompletedTrials) -> None:
    self._completed_trials.extend(delta.completed)

  def suggest(
      self, count: Optional[int] = None) -> Sequence[pyvizier.TrialSuggestion]:
    if count is None:
      return []
    completed_trial_ids = [t.id for t in self._completed_trials]
    current_index = max(completed_trial_ids)
    return [
        make_parameters(self._study_config, current_index + i)
        for i in range(count)
    ]

We can wrap `Designers` into Pythia Policies via the `DesignerPolicy` Wrapper and obtain a Pythia policy with exactly the behavior of `MyPolicy`.

In [ ]:
designer_factory = lambda study_config: MyDesigner(study_config)
MyPolicyCreator = lambda supporter: designer_policy.DesignerPolicy(
    supporter=supporter, designer_factory=designer_factory)

For a live example, see
[`EmukitDesigner`](https://github.com/google/vizier/blob/main/vizier/_src/algorithms/designers/emukit.py)
which implements a Bayesian Optimization GP-EI algorithm implemented in
[`emukit`](https://github.com/EmuKit/emukit), and can be wrapped by `DesignerPolicy`.

Also, see our [designer testing routine](https://github.com/google/vizier/blob/main/vizier/_src/algorithms/testing/test_runners.py) for an up-to-date example on how to interact with designers.

## Serializing Designers

Consider making the `Designer` serializable so that its state can be saved and
loaded. Vizier offers
[two options](https://github.com/google/vizier/blob/main/vizier/interfaces/serializable.py):

*   `Serializable` should be used if the entire algorithm state can be easily
    serialized and can be saved and restored in full.
*   `PartiallySerializable` should be used if the algorithm has subcomponents
    that are not easily serializable. The designer’s state can be recovered as
    long as it was initialized with the same arguments.

For an example of a `Serializable` object, see
[Population](https://github.com/google/vizier/blob/main/vizier/_src/algorithms/evolution/numpy_populations.py),
which is the internal state used by NSGA2.
[NSGA2 itself](https://github.com/google/vizier/blob/main/vizier/_src/algorithms/evolution/templates.py)
is only `PartiallySerializable` so that users can easily plug in their own
mutation and selection operations without worrying about serializations.

In [ ]:
nsga2_pythia_creator = lambda policy_supporter: designer_policy.PartiallySerializableDesignerPolicy(
    policy_supporter, nsga2.create_nsga2)

Serialization also makes a `Designer` run faster if its state size scales sublinearly in the number of observed Trials. For example, typical evolution algorithms and metaheuristics qualify, while GP-based algorithms do not because they use a non-parametric model. To do so, wrap a `(Partially)SerializableDesigner` into `(Partially)SerializableDesignerPolicy`, which takes care of the state management.